# Tensorflow1.0 에 있는 wide_n_deep_tutorial.py 를 분석해 보았습니다.

우선은 import 합니다.

In [ ]:
import argparse
import sys
import tempfile

from six.moves import urllib

import pandas as pd
import tensorflow as tf

원 소스에 보면 tempfile이 있으나 여기서는 사용하지 않을 것 입니다.

소스를 처음부터가 아닌 흐름을 파악할 수 있게 순서를 변경해서 보도록 하겠습니다.

## argument

In [ ]:
if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.register("type", "bool", lambda v: v.lower() == "true")
  parser.add_argument(
      "--model_dir",
      type=str,
      default="./model",
      help="Base directory for output models."
  )
  parser.add_argument(
      "--model_type",
      type=str,
      default="wide_n_deep",
      help="Valid model types: {'wide', 'deep', 'wide_n_deep'}."
  )
  parser.add_argument(
      "--train_steps",
      type=int,
      default=200,
      help="Number of training steps."
  )
  parser.add_argument(
      "--train_data",
      type=str,
      default="./data/training_data.csv",
      help="Path to the training data."
  )
  parser.add_argument(
      "--test_data",
      type=str,
      default="./data/test_data.csv",
      help="Path to the test data."
  )
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

* parser에 값들이 추가 되었고, 그 값들이 FLAGS에 들어간 점을 눈여겨 보세요.
* FLAG는 환경설정 값으로 계속해서 사용됩니다.
```
  parser.add_argument(
      "--model_dir",
      type=str,
      default="./model",
      help="Base directory for output models."
  )
```  
* argument로 --model_dir=디렉토리 이렇게 들어오다면 값이 '디렉토리'로 들어가고, argument가 없다면 default 값이 사용되는 것 입니다.
* 꺼내 쓸 때는 FLAG.model_dir 로 쓸 수 있습니다.

## Main

In [ ]:
FLAGS = None


def main(_):
  train_and_eval(FLAGS.model_dir, FLAGS.model_type, FLAGS.train_steps,
                 FLAGS.train_data, FLAGS.test_data)

* argument 에서 설정된 값이 train_and_eval 로 전달됩니다.

## train_and_eval

In [ ]:
def train_and_eval(model_dir, model_type, train_steps, train_data, test_data):

In [ ]:
"""Train and evaluate the model."""
  train_file_name, test_file_name = maybe_download(train_data, test_data)
  df_train = pd.read_csv(
      tf.gfile.Open(train_file_name),
      names=COLUMNS,
      skipinitialspace=True,
      engine="python")
  df_test = pd.read_csv(
      tf.gfile.Open(test_file_name),
      names=COLUMNS,
      skipinitialspace=True,
      skiprows=1,
      engine="python")

* maybe_download 는 데이터를 다운 받는 함수이고 이후에 나타납니다.
* pd는 panda라이브러리입니다. pd로 csv파일을 읽습니다.
* COLUMNS 는 문자들이 들어있는 배열입니다.
* csv파일에 comma(,)로 구분된 각 컬럼에 순서 대로 이름을 매칭시킵니다.

In [ ]:
# remove NaN elements
  df_train = df_train.dropna(how='any', axis=0)
  df_test = df_test.dropna(how='any', axis=0)

* [dropna](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html#) 에 전달된 값을 보면 axis가 0이거나 1일 수 있는데, 0이면 row단위로 없는 값을 제거 한 뒤 리턴하고, 1이면 column이 빈 것을 제거하로 리턴합니다.

In [ ]:
df_train[LABEL_COLUMN] = (
      df_train["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)

  df_test[LABEL_COLUMN] = (
      df_test["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)

* LABEL_COLUMN 에는 y값으로 사용할 컬럼명이 들어가 있습니다.
* panda (pd) 에서 리턴해준 df_train에서 이름이 "income_bracket" 것을 찾아 ">50K"가 있으면 1 없으면 0으로 값을 넣어 줍니다.

In [ ]:
model_dir = tempfile.mkdtemp() if not model_dir else model_dir

* 조건문입니다. 
* 만약 model_dir이 None이거나 False이면 tempfile.mkdtemp()를 호출하여 model_dir에 넣어주고, 그렇지 않다면 model_dir을 넣어 줍니다.
* 즉, model_dir이 None이거나 False인지를 체크해서 model_dir의 값을 넣어 줍니다.

In [ ]:
m = build_estimator(model_dir, model_type)

* 모델을 만들어 줍니다.

In [ ]:
m.fit(input_fn=lambda: input_fn(df_train), steps=train_steps)
  results = m.evaluate(input_fn=lambda: input_fn(df_test), steps=1)

* m.fit 모델을 학습 시킵니다. input_fn함수에 df_train을 넣어줍니다.
* m.evaluate는 학습된 모델을 평가합니다.

In [ ]:
results = m.evaluate(input_fn=lambda: input_fn(df_test), steps=1)
  for key in sorted(results):
    print("%s: %s" % (key, results[key]))

* 출력 값은 아래처럼 나왔습니다.
* accuracy: 0.818193
* accuracy/baseline_label_mean: 0.236226
* accuracy/threshold_0.500000_mean: 0.818193
* auc: 0.861029
* global_step: 202
* labels/actual_label_mean: 0.236226
* labels/prediction_mean: 0.32576
* loss: 1.24135
* precision/positive_threshold_0.500000_mean: 0.611251
* recall/positive_threshold_0.500000_mean: 0.632865

## build_estimator

In [ ]:
def build_estimator(model_dir, model_type):
  """Build an estimator."""
  # Sparse base columns.

  '''
  몇 개 안되는 값일 때... 즉 몇 가지 케이스로 정해진 경우
  '''
  gender = tf.contrib.layers.sparse_column_with_keys(column_name="gender",
                                                     keys=["female", "male"])

* sparse column들을 만드는데, 몇 가지 타입이 있습니다.
* [sparse_column_with_keys](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/sparse_column_with_keys) 는 값이 몇 가지로 한정되어 있을 때 사용합니다.

In [ ]:
'''
  해쉬 값으로 읽음.
  '''
  education = tf.contrib.layers.sparse_column_with_hash_bucket(
      "education", hash_bucket_size=1000)
  relationship = tf.contrib.layers.sparse_column_with_hash_bucket(
      "relationship", hash_bucket_size=100)
  workclass = tf.contrib.layers.sparse_column_with_hash_bucket(
      "workclass", hash_bucket_size=100)
  occupation = tf.contrib.layers.sparse_column_with_hash_bucket(
      "occupation", hash_bucket_size=1000)
  native_country = tf.contrib.layers.sparse_column_with_hash_bucket(
      "native_country", hash_bucket_size=1000)

* [sparse_column_with_hash_bucket](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/sparse_column_with_hash_bucket) 는 문자열로 된 것들을 hash 값으로 변경해서 사용합니다.

In [ ]:
# Continuous base columns.
  '''
  숫자 형.
  '''
  age = tf.contrib.layers.real_valued_column("age")
  education_num = tf.contrib.layers.real_valued_column("education_num")
  capital_gain = tf.contrib.layers.real_valued_column("capital_gain")
  capital_loss = tf.contrib.layers.real_valued_column("capital_loss")
  hours_per_week = tf.contrib.layers.real_valued_column("hours_per_week")

* [real_valued_column](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/real_valued_column)는 숫자형 데이터 입니다.

In [ ]:
# Transformations.
  age_buckets = tf.contrib.layers.bucketized_column(age,
                                                    boundaries=[
                                                        18, 25, 30, 35, 40, 45,
                                                        50, 55, 60, 65
                                                    ])

* 이 곳에서는 나이를 5살 단위로 만들어 줍니다. 

In [ ]:
# Wide columns and deep columns.
  wide_columns = [gender, native_country, education, occupation, workclass,
                  relationship, age_buckets,
                  tf.contrib.layers.crossed_column([education, occupation],
                                                   hash_bucket_size=int(1e4)),
                  tf.contrib.layers.crossed_column(
                      [age_buckets, education, occupation],
                      hash_bucket_size=int(1e6)),
                  tf.contrib.layers.crossed_column([native_country, occupation],
                                                   hash_bucket_size=int(1e4))]

* 위에서 만들어준 각 변수들을 배열에 넣어 줍니다.
* crossed_column 컬럼이 무엇인지 조사 필요 ******************************************************

In [ ]:
deep_columns = [
      tf.contrib.layers.embedding_column(workclass, dimension=8),
      tf.contrib.layers.embedding_column(education, dimension=8),
      tf.contrib.layers.embedding_column(gender, dimension=8),
      tf.contrib.layers.embedding_column(relationship, dimension=8),
      tf.contrib.layers.embedding_column(native_country,
                                         dimension=8),
      tf.contrib.layers.embedding_column(occupation, dimension=8),
      age,
      education_num,
      capital_gain,
      capital_loss,
      hours_per_week,
  ]

* embedding_column 이 무엇인지 조사 필요  ******************************************************

In [ ]:
if model_type == "wide":
    m = tf.contrib.learn.LinearClassifier(model_dir=model_dir,
                                          feature_columns=wide_columns)
  elif model_type == "deep":
    m = tf.contrib.learn.DNNClassifier(model_dir=model_dir,
                                       feature_columns=deep_columns,
                                       hidden_units=[100, 50])
  else:
    m = tf.contrib.learn.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=[100, 50]
        # ,fix_global_step_increment_bug=True
    )
  return m

* 여기서는 model_type이 wide_n_deep 이므로 else: 부분을 타게 됩니다.
* DNNLinearCombinedClassifier 가 무엇인지 조사 필요. ******************************************************

## input_fn

* input_fn은 train_and_eval에서 모델을 학습 시킬 때 input 값으로 들어가는 람다 함수 입니다.

In [ ]:
continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}

* CONTINUOUS_COLUMNS 에 들어있는 String 값을 하나씩 꺼내서 k로 가게 되고 key:value 형식의 데이터로 만들어 continuous_cols에 리턴 됩니다.
* 이 곳에는 숫자형 데이터만 들어 있습니다.

In [ ]:
categorical_cols = {
      k: tf.SparseTensor(
          indices=[[i, 0] for i in range(df[k].size)],
          values=df[k].values,
          dense_shape=[df[k].size, 1])
      for k in CATEGORICAL_COLUMNS}